In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

nav = webdriver.Chrome()

tabela_produtos = pd.read_excel("./buscas.xlsx")

display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [2]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
    # analisar se ele não tem nenhum termo banido
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    
    return tem_termos_banidos

def verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome):
    # analisar se ele tem TODOS os termos do nome do produto
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
        if palavra not in nome:
            tem_todos_termos_produtos = False
    
    return tem_todos_termos_produtos

def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    lista_ofertas = []

    # pesquisando pelo produto no google
    nav.get("https://www.google.com/")
    nav.find_element("xpath", '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/textarea').send_keys(produto, Keys.ENTER)

    # entrar na aba shopping
    elemento = nav.find_element("xpath", '//*[@id="hdtb-sc"]/div/div[1]/div[1]/div[2]/a')
    # elemento = nav.find_element("xpath", '/html/body/div[4]/div/div[5]/div/div[1]/div/div/div[1]/div/div/div/div/div[1]/div[2]/a')
    elemento.click()

    # pegando as informações dos produtos
    lista_resultados = nav.find_elements("class name", "i0X6df")

    for resultado in lista_resultados:
        nome = resultado.find_element("class name", "tAxDx").text
        nome = nome.lower()

        # analisar se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

        # analisar se ele tem TODOS os termos do nome do produto
        tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)

        # selecionar só os elementos que tem_termos_banidos = False e ao mesmo tempo tem_todos_termos_produtos = True
        if not tem_termos_banidos and tem_todos_termos_produtos:
            try:
                preco = resultado.find_element("class name", "a8Pemb").text
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)

                # se o preco está entre preco_minimo e preco_maximo
                if preco_minimo <= preco <= preco_maximo:
                    elemento_referencia = resultado.find_element('class name', 'bONr3b')
                    elemento_pai = elemento_referencia.find_element('xpath', '..') #pegando o elemento que vem logo antes dele ('pai')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
            except:
                continue
            
    return lista_ofertas

def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nome_produto = produto.split(" ")
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)
    lista_ofertas = []

    nav.get("https://www.buscape.com.br/")
    nav.find_element("xpath", '/html/body/div[1]/main/header/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)

    time.sleep(2)
    lista_resultados = nav.find_elements('class name', 'ProductCard_ProductCard_Inner__gapsh')

    for resultado in lista_resultados:
        try:
            preco = resultado.find_element('class name', 'Text_MobileHeadingS__HEz7L').text
            nome = resultado.find_element('class name', 'ProductCard_ProductCard_Name__U_mUQ').text
            nome = nome.lower()
            link = resultado.get_attribute('href')

            # analisar se ele não tem nenhum termo banido
            tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)

            # analisar se ele tem TODOS os termos do nome do produto
            tem_todos_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nome_produto, nome)

            if not tem_termos_banidos and tem_todos_termos_produtos:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco) # "+ impostos"
                
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome, preco, link))
        except:
            pass
    return lista_ofertas


In [3]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha,"Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]

    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    
    # verifica se as listas estão vazias
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
    else:
        tabela_google_shopping = None

    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape)
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
    else:
        tabela_buscape = None
    
display(tabela_ofertas)


,produto,preco,link
0,"iphone 12 64gb- branco, , color branco",3225.00,https://www.google.com/url?url=https://www.tra...
1,apple iphone 12 64gb 6.1 azul,3180.99,https://www.google.com/url?url=https://www.tra...
2,iphone 12 azul 64gb - seminovo,3190.00,https://www.google.com/url?url=https://www.cdk...
3,iphone 12 64gb | apple | iphone | usado | tama...,3115.00,https://www.google.com/url?url=https://www.enj...
4,apple iphone 12 64gb seminovo,3235.29,https://www.google.com/url?url=https://templod...
0,"pc gamer ryzen 5 5500, rtx 3060, 16gb ddr4, ss...",4311.30,https://www.google.com/url?url=https://www.eni...
1,"pc gamer w3, ryzen 5 5600g, geforce rtx 3060, ...",4499.90,https://www.google.com/url?url=https://www.loj...
2,"pc info3 gamer amd ryzen 5 rtx 3060 12gb, mem ...",4176.00,https://www.google.com/url?url=https://www.inf...
3,pc gamer amd ryzen 5 5500 16gb (rtx 3060 12gb)...,4329.90,https://www.google.com/url?url=https://www.sho...
4,"pc gamer arena games 6 intel i5 11400f, rtx 30...",4337.05,https://www.google.com/url?url=https://arenaco...


In [4]:
tabela_ofertas.to_excel('Ofertas.xlsx', index = False)